# Importamos las librerías necesarias para crear el flujo de trabajo

In [7]:
# Librería para manipular datasets
import pandas as pd

# Módulo para manipular paths
from pathlib import Path

# Librería identificar el encoding de los archivos 
import chardet

# Módulo para anipular expresiones regulares 
import re

# **EXTRACCIÓN**

Los archivos tienen diversos formatos y argumentos de importación. Para automatizarlo, creamos una función que considere todas estas consideraciones para instanciarlo con el path de cada file. 

In [8]:
def importar(path):
    with open(path, 'rb') as f:
        result = chardet.detect(f.read())
        # Para importar archivos csv
        if Path(path).suffix == ".csv": 
            data = pd.read_csv(path, encoding=result['encoding'], sep=None, engine='python', decimal='.')
        # Para importar archivos parquet
        elif Path(path).suffix == ".parquet": 
            data = pd.read_parquet(path, engine='pyarrow')
        # Para importar archivos json
        elif Path(path).suffix == ".json": 
            data = pd.read_json(path, precise_float=True)
        # Para importar archivos txt
        elif Path(path).suffix == ".txt": 
            data = pd.read_table(path, sep='|', engine='python')
            
    return data 

In [25]:
# Función para importar un excel con más de una pestaña
def importar_excel(path, sheet): 
  # Importamos solo una pestaña del file
  data = pd.read_excel(path,sheet_name=sheet)
  return data

Aplicamos las funciones para importar todos los datasets

In [21]:
# Archivo csv
sucursal_or = importar(r'C:\Users\USER\Documents\SOYHENRY\LABS\Repositorio\Datasets\1 sucursal.csv')
# Archivo parquet
producto_or = importar(r'C:\Users\USER\Documents\SOYHENRY\LABS\Repositorio\Datasets\2 producto.parquet')
# Archivo csv
precios_semana_20200413_or = importar(r'C:\Users\USER\Documents\SOYHENRY\LABS\Repositorio\Datasets\3 precios_semana_20200413.csv')   
# Archivo xlsx
precios_semanas_20200419_or = importar_excel(r'C:\Users\USER\Documents\SOYHENRY\LABS\Repositorio\Datasets\4 precios_semanas_20200419_20200426.xlsx', 'precios_20200426_20200426').copy()
# Archivo json
precios_semana_20200503_or = importar(r'C:\Users\USER\Documents\SOYHENRY\LABS\Repositorio\Datasets\5 precios_semana_20200503.json')
# Archivo txt
precios_semana_20200518_or = importar(r'C:\Users\USER\Documents\SOYHENRY\LABS\Repositorio\Datasets\6 precios_semana_20200518.txt')

**Nota**: Solo importamos la tabla de de la semana 20200426, debido a que la semana 20200419 no es representativa, queda pendiente consultar con el cliente para acordar el criterio correcto. 

Copiamos el dataset para evitar modificar los originales

In [351]:
# Archivo csv
sucursales = sucursal_or.copy()
# Archivo parquet
productos = producto_or.copy()
# Archivo csv
precios_20200413 = precios_semana_20200413_or.copy()
# Archivo xlsx
precios_20200426 = precios_semanas_20200419_or.copy()
# Archivo json
precios_20200503 = precios_semana_20200503_or.copy()
# Archivo txt
precios_20200518 = precios_semana_20200503_or.copy()

# **TRANSFORMACIÓN**

Analizamos cada uno de los datase para detectar que los datos sean reprsentativos 

**SUCURSALES**

In [352]:
# Visualizamos los datos
sucursales.head()

,id,comercioId,banderaId,banderaDescripcion,comercioRazonSocial,provincia,localidad,direccion,lat,lng,sucursalNombre,sucursalTipo
0,1-1-7,1,1,Super MAMI,Dinosaurio S.A.,AR-X,SALSIPUEDES,E53 1011 None,-31.126667,-64.295250,Super Mami 4,Hipermercado
1,10-1-1,10,1,Hipermercado Carrefour,INC S.A.,AR-B,San Isidro,Bernardo De Irigoyen 2647,-34.491345,-58.589025,San Isidro,Hipermercado
2,10-1-10,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Hurlingham,Av. Vergara 1910,-34.620610,-58.633769,Villa Tesei,Hipermercado
3,10-1-11,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Malvinas Argentinas,Av. Arturo Illia 3770,-34.528883,-58.701631,Malvinas Argentinas,Hipermercado
4,10-1-112,10,1,Hipermercado Carrefour,INC S.A.,AR-A,Salta,20 De Febrero 37,-24.789072,-65.413699,Salta,Hipermercado


In [353]:
# Utilizo punto y coma para evitar que me imprima el output
sucursales.rename(columns={'id':'sucursal_id', 'comercioId':'comercio_id', 'banderaId': 'bandera_id' }, inplace = True);

In [354]:
# Eliminamos datos duplicados
sucursales.drop_duplicates();

In [355]:
# Visualizamos los datos
sucursales.head()

,sucursal_id,comercio_id,bandera_id,banderaDescripcion,comercioRazonSocial,provincia,localidad,direccion,lat,lng,sucursalNombre,sucursalTipo
0,1-1-7,1,1,Super MAMI,Dinosaurio S.A.,AR-X,SALSIPUEDES,E53 1011 None,-31.126667,-64.295250,Super Mami 4,Hipermercado
1,10-1-1,10,1,Hipermercado Carrefour,INC S.A.,AR-B,San Isidro,Bernardo De Irigoyen 2647,-34.491345,-58.589025,San Isidro,Hipermercado
2,10-1-10,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Hurlingham,Av. Vergara 1910,-34.620610,-58.633769,Villa Tesei,Hipermercado
3,10-1-11,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Malvinas Argentinas,Av. Arturo Illia 3770,-34.528883,-58.701631,Malvinas Argentinas,Hipermercado
4,10-1-112,10,1,Hipermercado Carrefour,INC S.A.,AR-A,Salta,20 De Febrero 37,-24.789072,-65.413699,Salta,Hipermercado


In [356]:
sucursales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2333 entries, 0 to 2332
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   sucursal_id          2333 non-null   object 
 1   comercio_id          2333 non-null   int64  
 2   bandera_id           2333 non-null   int64  
 3   banderaDescripcion   2333 non-null   object 
 4   comercioRazonSocial  2333 non-null   object 
 5   provincia            2333 non-null   object 
 6   localidad            2333 non-null   object 
 7   direccion            2333 non-null   object 
 8   lat                  2333 non-null   float64
 9   lng                  2333 non-null   float64
 10  sucursalNombre       2333 non-null   object 
 11  sucursalTipo         2333 non-null   object 
dtypes: float64(2), int64(2), object(8)
memory usage: 218.8+ KB


**PRODUCTOS**

In [357]:
productos.head()

,id,marca,nombre,presentacion,categoria1,categoria2,categoria3
0,0000000001663,LA ANÓNIMA,Radicheta Atada La Anonima 1 Un,1.0 un,None,None,None
1,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un,None,None,None
2,0000000205870,SIN MARCA,Ojo de Bife 1 Kg,1.0 kg,None,None,None
3,0000000205894,SIN MARCA,Milanesa de Peceto Novillito 1 Kg,1.0 kg,None,None,None
4,0000000205955,SIN MARCA,Chiquizuela Novillito 1 Kg,1.0 kg,None,None,None


In [358]:
productos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72038 entries, 0 to 72037
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            72038 non-null  object
 1   marca         72036 non-null  object
 2   nombre        72036 non-null  object
 3   presentacion  72036 non-null  object
 4   categoria1    4 non-null      object
 5   categoria2    4 non-null      object
 6   categoria3    4 non-null      object
dtypes: object(7)
memory usage: 3.8+ MB


**Eliminar columnas categorias**

Nota: En la tabla de productos hay tres categorías que no son representativos de los datos, los eliminamos e informamos al cliente para que tome un criterio en la futuras fuentes de datos. 

In [359]:
# INFORME DE SITUACIÓN DE COLUMNAS CATEGORIES
print('*****INFORME DE SITUACIÓN DE COLUMNAS CATEGORIES*********')
print('Columna "categoria1"')
print('Datos nulos: {} '.format(len(productos[productos['categoria1'].isnull()])))
print('Datos únicos: {}'.format(pd.unique(productos['categoria1'])))

print('\n''Columna "categoria2"')
print('Datos nulos: {} '.format(len(productos[productos['categoria2'].isnull()])))
print('Datos únicos: {}'.format(pd.unique(productos['categoria2'])))

print('\n''Columna "categoria3"')
print('Datos nulos: {} '.format(len(productos[productos['categoria3'].isnull()])))
print('Datos únicos: {}'.format(pd.unique(productos['categoria3'])))

*****INFORME DE SITUACIÓN DE COLUMNAS CATEGORIES*********
Columna "categoria1"
Datos nulos: 72034 
Datos únicos: [None 'Perfumería y Cuidado Personal' 'Almacén' 'Alimentos Congelados']

Columna "categoria2"
Datos nulos: 72034 
Datos únicos: [None 'Cuidado Capilar' 'Aceites' 'Helados']

Columna "categoria3"
Datos nulos: 72034 
Datos únicos: [None 'Acondicionadores' 'Aerosol' 'Oliva' 'Helados']


In [360]:
productos = productos.drop(['categoria1', 'categoria2', 'categoria3'], axis=1)

In [361]:
# Eliminamos datos duplicados, usamos ; para no imprimir el output
productos.drop_duplicates();

In [362]:
# Verificamos si existe algún dato catacter no numérico en las filas del dataset
for i, row in enumerate(productos): 
    if r'/D' in row: 
        productos[i][row] = re.sub(r'\D', "", row)
        print(i, row)

In [363]:
# renombramos las columnas id para mantener el mismo criterio
productos.rename(columns={'id':'producto_id'}, inplace = True);

In [364]:
productos.head()

,producto_id,marca,nombre,presentacion
0,0000000001663,LA ANÓNIMA,Radicheta Atada La Anonima 1 Un,1.0 un
1,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un
2,0000000205870,SIN MARCA,Ojo de Bife 1 Kg,1.0 kg
3,0000000205894,SIN MARCA,Milanesa de Peceto Novillito 1 Kg,1.0 kg
4,0000000205955,SIN MARCA,Chiquizuela Novillito 1 Kg,1.0 kg


Existen dos datos nulos con id asignado, los eliminamos 

In [365]:
productos[productos['nombre'].isnull()]

,producto_id,marca,nombre,presentacion
53619,7798024299045,None,None,None
55798,7798061190213,None,None,None


In [366]:
productos = productos.drop([53619, 55798], axis=0).reset_index()

**PRECIOS**

Hay varios dataset que representan la misma información, por tanto limpiaremos los datos y exportaremos solo un dataset con la concatenación de todos

In [367]:
print('\n','precios_20200413''\n', precios_20200413.head())
print('\n','precios_20200426''\n', precios_20200426.head())
print('\n','precios_20200503''\n', precios_20200503.head())
print('\n','precios_20200518''\n', precios_20200518.head())


 precios_20200413
    precio    producto_id sucursal_id
0   29.90  0000000001663     2-1-014
1   29.90  0000000002288     2-1-032
2   39.90  0000000002288     2-1-096
3  499.99  0000000205870     9-1-686
4  519.99  0000000205870     9-2-248

 precios_20200426
     precio sucursal_id  producto_id
0    399.0     2-1-092       2288.0
1    299.0     2-1-206       2288.0
2    399.0     2-2-241       2288.0
3  49999.0     9-1-430     205870.0
4  53999.0       9-2-4     205870.0

 precios_20200503
    precio    producto_id sucursal_id
0    29.9  0000000002288     2-1-187
1    39.9  0000000002288     2-3-247
2  499.99  0000000205870     9-1-685
3  539.99  0000000205870      9-2-22
4  519.99  0000000205870      9-2-59

 precios_20200518
    precio    producto_id sucursal_id
0    29.9  0000000002288     2-1-187
1    39.9  0000000002288     2-3-247
2  499.99  0000000205870     9-1-685
3  539.99  0000000205870      9-2-22
4  519.99  0000000205870      9-2-59


La tabla de preccios precios_20200426 tiene un orden de columnas diferentes, por tanto la colocamos en la misma posición que el resto de los datasets

**Analizamos los datos nulos**

**Nota**: en el dataset precios hay tres columnas: precio, sucursal_id y producto_id, si alguno de estos datos falta entonces la infomación de la fila no tienen utilidad, bajo este criterio eliminamos las filas con al menos un dato nulo. 

In [368]:
print('REPORTE DE PORCENTAJE DE DATOS NULOS EN LAS TABLAS DE PRECIOS')
print('precios_20200413 =', round((len(precios_20200413[precios_20200413['producto_id'].isnull()]) / len(precios_20200413['producto_id']))*100,4), '%')
print('precios_20200426 =', round((len(precios_20200426[precios_20200426['producto_id'].isnull()]) / len(precios_20200426['producto_id']))*100,4), '%')
print('precios_20200503 =', round((len(precios_20200503[precios_20200503['producto_id'].isnull()]) / len(precios_20200503['producto_id']))*100,4), '%')
print('precios_20200518 =', round((len(precios_20200518[precios_20200426['producto_id'].isnull()]) / len(precios_20200518['producto_id']))*100,4), '%')

REPORTE DE PORCENTAJE DE DATOS NULOS EN LAS TABLAS DE PRECIOS
precios_20200413 = 0.0032 %
precios_20200426 = 2.8229 %
precios_20200503 = 0.0 %
precios_20200518 = 2.7259 %


C:\Users\USER\AppData\Local\Temp\ipykernel_37936\2949426603.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print('precios_20200518 =', round((len(precios_20200518[precios_20200426['producto_id'].isnull()]) / len(precios_20200518['producto_id']))*100,4), '%')


In [369]:
# eliminamos los datos nulos
precios_20200413.dropna(how='any', inplace=True)
precios_20200426.dropna(how='any', inplace=True)
precios_20200503.dropna(how='any', inplace=True)
precios_20200518.dropna(how='any', inplace=True)

In [370]:
# eliminamos datos duplicados
precios_20200413.drop_duplicates()
precios_20200426.drop_duplicates()
precios_20200503.drop_duplicates()
precios_20200518.drop_duplicates();

In [371]:
# Eliminamos espacios vacíos
def eliminar_vacios(dataset): 
    columnas = dataset.columns.to_list()

    for columna in columnas: 
        for i, row in enumerate(dataset):
            if row == str: 
                dataset[columna] = dataset[columna].strip()
            if row == '': 
                dataset = dataset.drop(axis=0, inplace=True)
    return dataset


In [372]:
eliminar_vacios(precios_20200413)
eliminar_vacios(precios_20200426)
eliminar_vacios(precios_20200503)
eliminar_vacios(precios_20200518);

In [373]:
def reorderar_columnas(dataset): 
  # Extraemos todas las columnas del dataset
  columnas = dataset.columns.to_list()
  for columna in columnas: 
    # Eliminamos cada columna
    sucursal_id = dataset.pop('sucursal_id') 
    producto_id = dataset.pop('producto_id') 
    precio =  dataset.pop('precio') 
    # Las insertamos nuevamente en el orden indicado
    dataset.insert(0, 'sucursal_id', sucursal_id) 
    dataset.insert(1, 'producto_id', producto_id)
    dataset.insert(2, 'precio', precio)
  
  return dataset


In [374]:
# Instanciamos la función en todos los datasets
reorderar_columnas(precios_20200413)
reorderar_columnas(precios_20200426)
reorderar_columnas(precios_20200503)
reorderar_columnas(precios_20200518);

In [375]:
precios_20200413['precio'].sort_values().astype(int)

183693        0
183186        0
183694        0
183952        0
183951        0
          ...  
22045     30528
22337     31000
30972     32469
22336     32860
30971     34417
Name: precio, Length: 472151, dtype: int32

In [376]:
precios_20200426['precio'].astype(int)

0           399
1           299
2           399
3         49999
4         53999
          ...  
478904    13999
478905     3499
478906     3125
478907     3125
478908     1989
Name: precio, Length: 463753, dtype: int32

In [377]:
for i, row in enumerate(precios_20200503['precio']): 
    if row == '': 
        precios_20200503 =  precios_20200503.drop(i, axis=0)

ValueError: cannot insert level_0, already exists

In [ ]:
for i, row in enumerate(precios_20200503['precio']): 
    if row == '': 
        precios_20200503 =  precios_20200503.drop(i, axis=0)

In [ ]:
for i, row in enumerate(precios_20200503['precio']): 
    if row == '': 
        print(i)
        break

352024


In [ ]:
# Concatenamos todos los dataset de precios
precios = pd.concat([precios_20200413, precios_20200426, precios_20200503, precios_20200518], axis=0)

In [ ]:
for i in range(len(precios['sucursal_id'])):
    precios['sucursal_id'][i] = re.sub(r'\D', "", str(int(precios['sucursal_id'][i])))

for i in range(len(precios['producto_id'])):
    precios['producto_id '][i] = re.sub(r'\D', "", str(int(precios['producto_id '][i])))

precios['sucursal_id'] = precios['sucursal_id'].astype(int)
precios['producto_id'] = precios['producto_id'].astype(int)
precios['precio'] = precios['precio'].astype(float)

TypeError: cannot convert the series to <class 'int'>